In [ ]:
import speasy as spz
from datetime import datetime
import matplotlib.pyplot as plt

Plot Solar Orbiter observation data between 2020/07/23 16:00:00 and 2020/07/24 06:00:00.

In [ ]:
start = "2020/07/24T02:00:00"
stop = "2020/07/24T04:00:00"

In [ ]:
solo_b_id = "solo_b_rtn_hr"
solo_n_id = "pas_momgr_n"
solo_v_id = "pas_mom_v"

parameter_ids = [solo_b_id, solo_n_id, solo_v_id]
parameter_data = [spz.get_data(f"amda/{pid}", start, stop) for pid in parameter_ids]

In [ ]:
n = len(parameter_data)
fig, ax = plt.subplots(n,1, figsize=(10, n*3), sharex=True)
for i, param in enumerate(parameter_data):
    t = [datetime.utcfromtimestamp(ti) for ti in param.time]
    #ax[i].plot(param.time, param.data)
    ax[i].plot(t, param.data)
    ax[i].set_ylabel(parameter_ids[i])
import matplotlib.dates as mdates
datefmt = mdates.DateFormatter("%H:%M")

ax[-1].xaxis.set_major_formatter(datefmt)
plt.show()

In [ ]:
fig, ax = plt.subplots(4,1, sharex=True, figsize=(10, 8))

from scipy.interpolate import interp1d
import numpy as np
import pandas as pd

b_data = parameter_data[0]

b_func = interp1d(b_data.time, b_data.data, axis=0)
t = [datetime.utcfromtimestamp(ti) for ti in b_data.time]
nt = np.arange(b_data.time[0], b_data.time[-1], 1)

b = b_func(nt)

b_ts = pd.DataFrame(data = b, index=nt)

b_mean = b_ts.rolling(20).mean()
b_mean_func = interp1d(b_mean.index-600, b_mean.values, axis=0, bounds_error=False)
bm = b_mean_func(nt)


ax[0].plot(t, b_data.data)
ax[0].set_ylabel("b (RTN)")

tt = [datetime.utcfromtimestamp(ti-600) for ti in b_mean.index]
ax[1].plot(tt, b_mean.values)
ax[1].set_ylabel("b_mean (RTN)")

# one arrow per 10 minute
x_bases = np.arange(b_data.time[0], b_data.time[-1], 60)
for x_b in x_bases:
    bt = b_func(x_b)
    bmt = b_mean_func(x_b)
    db = bt - bmt
    db[0] = .001 * db[0]
    ax[2].arrow(datetime.utcfromtimestamp(x_b), 0, db[0], db[1], 
               width=.0001,
               head_width=.0003, 
               shape="full",
               head_length=30*.0003)
               #length_includes_head=False)
    ax[3].arrow(datetime.utcfromtimestamp(x_b), 0, db[0], db[2])
               #width=.0001,)
               #length_includes_head=False)
ttt = [datetime.utcfromtimestamp(ti) for ti in nt]
#ax[2].plot(ttt, b-bm)
ax[3].xaxis.set_major_formatter(datefmt)
ax[3].set_xlabel("Time")
ax[3].set_xlim(t[0], t[-1])
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
plt.plot(b_mean.values)